# LS solver

*Advanced Econometrics 2016-2017. Case 1, least squares (LS) implementation.*

## Problem
Ordinary Least Squares (OLS) is a solution for a system of linear equations by minimizing the sum of squared differences between the observed values ($y$) and the corresponding modelled values ($\hat y = X \hat\beta$).

A system of equations is given by
$$ y_i = \beta_1 + \beta_2 X_{i2}  + \beta_3 X_{i3}  + ... + \beta_K X_{iK} + \mu_i,  \quad i=1,...,N$$

or in matrix notation
$$ y = X \beta + \mu$$

with $y, \mu \in \mathbb{R}^{N\times1}$, $X \in \mathbb{R}^{N\times K}$ and $\beta \in \mathbb{R}^{K\times 1}$.

The OLS solves
$$\operatorname{\,min} \, \big\|y - X \hat\beta \big\|^2$$
that with some algebra leads to the [normal equations](https://en.wikipedia.org/wiki/Normal_equations) 
$$ X' X\beta = X'y$$ 

## Solution

### Normal equations

In case of a [rank][1] complete matrix $X$ the normal equations can be solved by inverting the ([Gram](https://en.wikipedia.org/wiki/Gramian_matrix)) matrix $X'X$ such that $\hat\beta = (X'X)^{-1}Xy$.

As X'X is a symmetric, positive definite matrix it's computationally more efficient to use the cholesky decomposition of $X'X = LL'$ with $L$ a lower triangular matrix. This gives $LL'\hat\beta = X'y$. First solve $Lz = X'y$ for $z$ by [forward substitution](https://en.wikipedia.org/wiki/Forward_substitution) and then $L'\hat\beta = z$ for $\beta$ by backward substitution.

However, solving the normal equations is numerically unstable, i.e. it is very sensitive to small pertubations in $X$. The [condition number](https://en.wikipedia.org/wiki/Condition_number) $\kappa$ of the system is worsened: $\kappa(X'X) = [\kappa(X)]^2$. We show the impact below.

[1]: https://en.wikipedia.org/wiki/Rank_(linear_algebra)

We first simulate some parameters and data using the same notation as in the slides.

In [29]:
K = 5   # number of parameters
N = 100 # number of observations

# Create actual parameters and observations
β = randn(K)
X = randn(N, K)
X[:,1] = ones(N) # intercept
σ = 0.1
μ = randn(N) * σ # ~ N(0,1)
y = X * β + μ;

In [2]:
#define function
OLS_normal(y, X) = inv(X'X) * X'y

# test and print result side by side for quick sanity check
β̂_normal = OLS_normal(y, X)
hcat(β̂_normal, β)

5×2 Array{Float64,2}:
  1.39878    1.39357 
 -0.230888  -0.22643 
 -1.35612   -1.3652  
  0.538232   0.551479
 -0.218218  -0.219672

In [4]:
function OLS_chol(y, X)
    X′X_chol = cholfact(X'X)
    X′X_chol\(X'*y)
end

# test and print result side by side for quick sanity check
β̂_chol = OLS_chol(y, X)
hcat(β̂_chol, β)

5×2 Array{Float64,2}:
  1.39878    1.39357 
 -0.230888  -0.22643 
 -1.35612   -1.3652  
  0.538232   0.551479
 -0.218218  -0.219672

### QR Factorization
We decompose $X$ to its orthogonal [QR decomposition](https://en.wikipedia.org/wiki/QR_decomposition):
$$X = Q\begin{bmatrix}
R \\
0
\end{bmatrix} $$
with $Q\in\mathbb{R}^{N\times N} $ an [orthogonal matrix](https://en.wikipedia.org/wiki/Orthogonal_matrix) and $R\in\mathbb{R}^{k\times k}$ an upper triangular matrix (with positive diagonal elements).
The solution is then given by
$$R \hat\beta =\left(Q' \mathbf y \right)_K$$

This is the default method in Julia for solving OLS (for non-square matrix $X$) with `β̂ = X\y`

In [26]:
function OLS_qr(y, X)
    X_qr = qrfact(X)
    X_qr \ y
end


OLS_qr (generic function with 1 method)

In [36]:
β̂_qr = OLS_qr(y, X)
hcat(β̂_qr, β)

5×2 Array{Float64,2}:
 -0.959486   -0.97892  
 -1.50845    -1.50726  
  0.0682501   0.0591163
  0.645577    0.640026 
  0.520167    0.504102 

### SVD
Another orthogonal decomposition uses the [singular value decomposition](https://en.wikipedia.org/wiki/Singular_value_decomposition) (SVD) of $X$:
$$X = U \Sigma V'$$
with both $U \in\mathbb{R}^{N\times N}$ and $V \in\mathbb{R}^{K\times K}$ an orthogonal matrix, and $\Sigma\in\mathbb{R}^{N\times K}$ a diagonal matrix. SVD can transform any matrix into a diagonal matrix with the right choice of orthogonal coordinate systems for its domain and range. This even works for rand deficient matrices and is numerically stable! In such an ill-conditioned system, SVD will return the solution $\hat\beta$ that has the smallest norm.




In [37]:
function OLS_svd(y, X)
    X_svd = svdfact(X)
    X_svd \ y
end

OLS_svd (generic function with 1 method)

In [44]:
β̂_svd = OLS_svd(y, X)
hcat(β̂_svd, β)

5×2 Array{Float64,2}:
 -0.959486   -0.97892  
 -1.50845    -1.50726  
  0.0682501   0.0591163
  0.645577    0.640026 
  0.520167    0.504102 

## Numerical Stability
We now test numerical stability (example 3.5 in [1]) with 
$$X^s = \begin{bmatrix}
1 & 1\\
1 & 1+\epsilon
\end{bmatrix}, 
y^s = \begin{bmatrix}
2 \\
2 
\end{bmatrix}$$
This gives the solution $\hat\beta=\begin{bmatrix}2 & 0\end{bmatrix}'$ .

This system is ill-conditioned because a slight perturbation in $$y^s_e = \begin{bmatrix}
2 \\
2 + \epsilon
\end{bmatrix}$$
gives the compeltely different solution $\hat\beta=\begin{bmatrix}1 & 1\end{bmatrix}'$

We show that methods using the normal equations do not find the second solution.

[1]: Numerically Efficient Methods for Solving Least Squares Problems, Do Q Lee.

In [39]:
ϵ = 1e-7
Xˢ = [1 1;1 1+ϵ]
yˢ = [2; 2]
OLS_normal(yˢ, Xˢ)

2-element Array{Float64,1}:
 2.0
 0.0

In [50]:
yˢₑ = [2; 2+ϵ]
@show OLS_normal(yˢₑ, Xˢ)
@show OLS_chol(  yˢₑ, Xˢ);

OLS_normal(yˢₑ,Xˢ) = [0.75,1.125]
OLS_chol(yˢₑ,Xˢ) = [0.727273,1.27273]


In [52]:
@show OLS_qr( yˢₑ, Xˢ)
@show OLS_svd(yˢₑ, Xˢ);

OLS_qr(yˢₑ,Xˢ) = [1.0,1.0]
OLS_svd(yˢₑ,Xˢ) = [1.0,1.0]


### Advanced methods
* [James-Stein estimator](https://en.wikipedia.org/wiki/James%E2%80%93Stein_estimator)
* 

In [53]:
using IterativeSolvers

INFO: Recompiling stale cache file /Users/ken/.julia/lib/v0.5/IterativeSolvers.ji for module IterativeSolvers.


In [77]:
lsqr(X, y)

([-0.959486,-1.50845,0.0682501,0.645577,0.520167],IterativeSolvers.ConvergenceHistory{Tuple{Float64,Float64,Float64},Array{Float64,1}}(true,(1.4901161193847656e-8,1.4901161193847656e-8,6.7108864e7),12,[3.59688,1.11312,1.06635,1.0658,1.0658]))

In [71]:
using PlotRecipes

In [72]:
plot(ch)

LoadError: In convertToAnyVector, could not handle the argument types: (IterativeSolvers.ConvergenceHistory{Tuple{Float64,Float64,Float64},Array{Float64,1}},)

In [63]:
Pkg.add("IterativeSolvers")

LoadError: unsatisfiable package requirements detected: no feasible version could be found for package: MultiScaleArrays
  (you may try increasing the value of the
   JULIA_PKGRESOLVE_ACCURACY environment variable)
 in resolve(::Dict{String,Base.Pkg.Types.VersionSet}, ::Dict{String,Dict{VersionNumber,Base.Pkg.Types.Available}}) at /Applications/Julia-0.5.app/Contents/Resources/julia/lib/julia/sys.dylib:? (repeats 2 times)
 in resolve(::Dict{String,Base.Pkg.Types.VersionSet}, ::Dict{String,Dict{VersionNumber,Base.Pkg.Types.Available}}, ::Dict{String,Tuple{VersionNumber,Bool}}, ::Dict{String,Base.Pkg.Types.Fixed}) at ./pkg/entry.jl:476
 in edit(::Function, ::String, ::Base.Pkg.Types.VersionSet, ::Vararg{Base.Pkg.Types.VersionSet,N}) at ./pkg/entry.jl:30
 in (::Base.Pkg.Entry.##2#5{String,Base.Pkg.Types.VersionSet})() at ./task.jl:360

hat can be solved by the (Moore-Penrose) [pseudo-inverse](https://en.wikipedia.org/wiki/Moore–Penrose_pseudoinverse) $X^+$ such that $\hat\beta = X^+ y$. 



In [ ]:
function LS(y, X, )

In [178]:
# |β̂_naive - β| ~ σ, √K, 1/√N
norm(β̂_naive - β),  σ * √K / √N # up to constant≈3

(0.20406622789422657,0.05)